In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
'''for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))'''

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

"for dirname, _, filenames in os.walk('/kaggle/input'):\n    for filename in filenames:\n        print(os.path.join(dirname, filename))"

In [1]:
import os
import shutil

def merge_folders(new_folder_name, *folders_to_merge):
    if not os.path.exists(new_folder_name):
        os.makedirs(new_folder_name)

    for folder in folders_to_merge:
        for item in os.listdir(folder):
            s = os.path.join(folder, item)
            d = os.path.join(new_folder_name, item)
            if os.path.isdir(s):
                if item.lower() in os.listdir("/kaggle/working/"+new_folder_name):
                    files = os.listdir(s)
                    
                    for file in files:
                        d = d.lower()
                        j = os.path.join(s, file)
                        k = os.path.join(d, file)
                        k = "/kaggle/working/"+k
                        if file in os.listdir(d):
                            nw = k.replace('.','a.')
                            os.rename(k ,nw)
                            shutil.copy2(j,k)
                        else:
                            shutil.copy2(j,k)          
                else:
                    shutil.copytree(s, d)
            else:
                shutil.copy2(s, d)

shutil.rmtree("final_crop")

In [3]:
merge_folders("final_crop","/kaggle/input/agriculture-crop-images/crop_images","/kaggle/input/agriculture-crop-images/kag2","/kaggle/input/agriculture-crop-images/some_more_images/some_more_images")

os.listdir("/kaggle/working/final_crop")

len(os.listdir("/kaggle/working/final_crop/wheat"))

shutil.rmtree("final_crop")

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [8]:
data_gen = ImageDataGenerator(zoom_range=0.2,
                              shear_range=0.3,
                              rotation_range=90,
                              vertical_flip= True,
                              horizontal_flip=True,
                              rescale=1/255)

In [9]:
train_data1 = data_gen.flow_from_directory('/kaggle/working/final_crop',
                             target_size=(224,224), batch_size=32)

Found 1055 images belonging to 5 classes.


train_data2 = data_gen.flow_from_directory('../input/agriculture-crop-images/kag2',
                             target_size=(224,224), batch_size=32)

In [10]:
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten
from tensorflow.keras.models import Sequential

In [11]:
from tensorflow.keras.applications.vgg16 import VGG16

In [12]:
vgg=VGG16(include_top=False, input_shape=(224,224,3))

2023-01-17 19:43:46.809689: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-17 19:43:46.896489: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-17 19:43:46.897361: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-17 19:43:46.898620: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

58900480/58889256 [==============================] - 3s 0us/step


In [26]:
from tensorflow.keras.models import Model

for layer in vgg.layers:
    layer.trainable=False
    
hidden_layer = Dense(1024,activation='relu')(Flatten()(vgg.output))
hidden_layer1 = Dense(512,activation='relu')(hidden_layer)
hidden_layer2 = Dense(256,activation='relu')(hidden_layer1)
hidden_layer3 = Dense(128,activation='relu')(hidden_layer2)
hidden_layer4 = Dense(64,activation='relu')(hidden_layer3)
final_layer=Dense(5,activation='softmax')(hidden_layer4)

vgg_model=Model(inputs=vgg.input, outputs=final_layer)
vgg_model.summary()

In [27]:
vgg_model=Model(inputs=vgg.input, outputs=final_layer)
vgg_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [28]:
vgg_model.compile(loss='categorical_crossentropy', metrics=['accuracy'])
vgg_model.fit_generator(train_data1,epochs=40)

Epoch 1/40
33/33 [==============================] - 15s 424ms/step - loss: 3.4837 - accuracy: 0.2152
Epoch 2/40
33/33 [==============================] - 14s 412ms/step - loss: 1.5347 - accuracy: 0.3355
Epoch 3/40
33/33 [==============================] - 14s 423ms/step - loss: 1.3956 - accuracy: 0.3924
Epoch 4/40
33/33 [==============================] - 14s 416ms/step - loss: 1.3419 - accuracy: 0.4597
Epoch 5/40
33/33 [==============================] - 14s 431ms/step - loss: 1.2628 - accuracy: 0.5118
Epoch 6/40
33/33 [==============================] - 14s 410ms/step - loss: 1.1674 - accuracy: 0.5441
Epoch 7/40
33/33 [==============================] - 14s 423ms/step - loss: 1.0736 - accuracy: 0.6028
Epoch 8/40
33/33 [==============================] - 14s 418ms/step - loss: 0.8851 - accuracy: 0.6493
Epoch 9/40
33/33 [==============================] - 14s 426ms/step - loss: 0.8528 - accuracy: 0.6957
Epoch 10/40
33/33 [==============================] - 14s 411ms/step - loss: 0.7681 - accura

In [29]:
train_data1.class_indices

{'jute': 0, 'maize': 1, 'rice': 2, 'sugarcane': 3, 'wheat': 4}

In [30]:
import cv2

In [31]:
def predict(path):
    img=cv2.imread(path)
    img=cv2.resize(img,(224,224))/255
    y=vgg_model.predict_on_batch(img.reshape(-1,224,224,3)).argmax()
    if y==0:
        return 'jute'
    if y==1:
        return 'maize'
    if y==2:
        return 'rice'
    if y==3:
        return 'sugarcane'
    if y==4:
        return 'wheat'

In [32]:
def accuracy(path):
    imgs=os.listdir(path)
    true = []
    false = []
    for img in imgs:
        yp=predict(path+img)
        if yp.upper() in img.upper():
            true.append('a')
        else:
            false.append('b')
    return len(true)/(len(true)+len(false))
    

In [33]:
accuracy('/kaggle/input/agriculture-crop-images/test_crop_image/')

0.5098039215686274

In [34]:
accuracy('/kaggle/input/test-crop/crop/')

0.6666666666666666